In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import HTML
import tqdm
import sln_letter_fit
from sln_letter_fit import FitParams, OptimizationLoggingParams
import loader, plotting

%load_ext autoreload
%autoreload 1
%aimport sln_letter_fit, loader, plotting

# Conventions:

* Trajectory - full set of points from pen down to pen up
* Stroke - subset of trajectory segmented by velocity curve
* Segment - either a stroke or trajectory, depending on context

# Fit and Plot the letter 'D'

In [ ]:
# Don't use convenience function, so that we can see more clearly what's going on
strokes = loader.load_segments('D', 1)
sol, history, fitter, _ = sln_letter_fit.fit_trajectory(
    strokes,
    fit_params=FitParams(max_iters=50, initialization_strategy_params=' :D '),
    optimization_logging_params=OptimizationLoggingParams(log_optimization_values=False,
                                                          progress_bar_class=tqdm.tqdm_notebook))
fig, ax = plt.subplots()
plotting.plot_trajectory(ax, strokes, sol, iteration_number=len(history))
plt.show()

In [ ]:
# Use convenience function - it should give the same result
fig, ax = plt.subplots()
_, anim = sln_letter_fit.fit_and_plot_trajectories(ax,
                                                   'D',
                                                   num_strokes=None,
                                                   max_iters=50,
                                                   animate=True)
plt.close()
HTML(anim.to_jshtml())

# Playing around

In [ ]:
# Plot the hole in the letter "A"
_, anim = sln_letter_fit.fit_and_plot_trajectories(ax,
                                                   'A',
                                                   num_strokes=3,
                                                   trajectory_indices=[1],
                                                   max_iters=100,
                                                   animate=True)
HTML(anim.to_jshtml())


In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))
_, anim = sln_letter_fit.fit_and_plot_trajectories(
    ax,
    'C',
    num_strokes=None,
    trajectory_indices=(0,),
    max_iters=250,
    log_history=True,
    animate=True,
    animation_oversample=1,
    save_animation_fname='data/optimization_animation_C_allstrokes.mp4')
plt.close()
HTML(anim.to_jshtml())

# Fit and plot lots of letters

In [ ]:
def fit_and_plot_letters(letters, **kwargs):
    fig, axes = plt.subplots(1, len(letters), figsize=(25, 5))
    for ax, letter in zip(axes, letters):
        sln_letter_fit.fit_and_plot_trajectories(ax, letter, **kwargs) # animate doesn't work
        ax.axis('equal')

In [ ]:
fit_and_plot_letters('ABCD', num_strokes=None, trajectory_indices=None, max_iters=250)

In [ ]:
# initializations:
fit_and_plot_letters('ABCD', num_strokes=None, trajectory_indices=None, max_iters=0)

In [ ]:
animation_oversample = 10
for letter in 'ABCDE':
    save_animation_fname = 'data/optimization_animation_{:}_allstrokes_fast.mp4'.format(letter)
    fig, ax = plt.subplots(figsize=(5, 4))
    _, anim = fit_and_plot_trajectories(ax,
                                        letter,
                                        num_strokes=None,
                                        # trajectory_indices=(0,),
                                        trajectory_indices=None,
                                        max_iters=250,
                                        log_history=True,
                                        animate=True)
    save_animation_fname = None
    plt.close()

In [92]:
# Combine all letters into a single video
import cv2

w_, h_ = 500, 400
w, h = w_ * 7, h_ * 4
T = 101
imgs = np.ones((T, h, w, 3), dtype=np.uint8) * 255

for i, letter in enumerate(tqdm.notebook.tqdm('ABCDEFGHIJKLMNOPQRSTUVWXYZ')):
    xi, yi = i % 7, i // 7
    x, y = xi * w_, yi * h_
    vid = cv2.VideoCapture('data/optimization_animation_{:}_allstrokes_fast.mp4'.format(letter))
    for k in range(T):
        success, frame = vid.read()
        if success:
            imgs[k, y:y + h_, x:x + w_, :] = frame

fps = 30
out = cv2.VideoWriter('data/optimization_animation_all_allstrokes_fast.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h), True)
for frame in imgs:
    out.write(frame)
out.release()
out = None

  0%|          | 0/26 [00:00<?, ?it/s]